In [1]:
import keras
from keras.models import Model
from keras.layers import Flatten, Dense, Dropout
from keras.layers.convolutional import MaxPooling2D, Convolution2D, AveragePooling2D

#base_model = keras.applications.inception_resnet_v2.InceptionResNetV2(include_top=False, weights='imagenet',input_shape=(299,299,3),pooling='avg')
base_model = keras.applications.inception_resnet_v2.InceptionResNetV2(include_top=False, weights='imagenet',input_shape=(299,299,3))

for layer in base_model.layers:
    layer.trainable = False


Using TensorFlow backend.


In [3]:
base_model.summary()

In [3]:
x = base_model.output
x = Convolution2D(384, 1, 1, activation='relu', border_mode='same')(x)
x = Flatten()(x)
x = Dense(4096, activation='relu', name='fc1')(x)
x = Dropout(0.8)(x)
x = Dense(4096, activation='relu', name='fc2')(x)
x = Dropout(0.8)(x)
predictions = Dense(228, activation='sigmoid', name='predictions')(x)

model = Model(input=base_model.input, output=predictions)

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:2: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(384, (1, 1), activation="relu", padding="same")`
  
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:10: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("pr...)`
  # Remove the CWD from sys.path while we load stuff.


In [4]:
model.summary()

In [5]:
###################切分数据--1225029：   以及尝试训练
#####数据分析
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
from PIL import Image

import tensorflow as tf 
from sklearn.model_selection import train_test_split #数据拆分
from keras.models import load_model
from keras.optimizers import Adam,SGD

#model = create_inception_v4()
#model=create_inception_resnet_v2(nb_classes=228)
#model.summary()
#model = load_model('temp_vgg.h5')
#model.compile(loss = 'binary_crossentropy',optimizer='adam',metrics=['accuracy'])


train_data = pd.read_csv('data/train.csv')   ####1014544
val_data = pd.read_csv('data/val.csv')
#test_data = pd.read_csv('data/test.csv')
file_dir="data/train_data/"
val_dir="data/val_data/"

step=10047 ##2511,10047
BATCH_SIZE=16 ##16,32
lr_power = 0.9
lr_base=0.001



###########----------------------validationData
images = []
labels = []
for img_num in range(0,3000):
    img_name=val_data['filename'][img_num]
    img_path = val_dir+img_name
    if os.path.exists(img_path):
        im = Image.open(img_path).convert('RGB') #转换为灰度图，选项:1,L,P,RGB,RGBA,CMYK,YCbCr,I,F
        imResize = im.resize((299, 299))
        imageData = np.array(imResize)
        images.append(imageData)
        #(filename,extension) = os.path.splitext(img_name)

        #labels.append(train_data['landmark_id'][img_num])
        temp=img_name.split("_")
        temp=temp[3].split(".")
        temp=temp[0].split("[")
        temp=temp[1].split("]")
        temp=temp[0].split(",")

        each_label_list = [0] * 228
        for j in range(0,len(temp)):
            each_label_list[int(temp[j])-1]=1

        labels.append(each_label_list)

X_val = np.array(images)
X_val = X_val.reshape(X_val.shape[0],299,299,3).astype('float32')
y_val=np.array(labels,dtype=np.float)

###########------------------------------------------

for epoch_t in range(0,20):
    lr_now = lr_base * ((1 - float(epoch_t)/20) ** lr_power)
    #def_adam=Adam(lr=lr_now)
    optimizer = SGD(lr=lr_now, momentum=0.9)
    model.compile(loss = 'binary_crossentropy',optimizer=optimizer,metrics=['accuracy'])
    
    img_range_high=0
    for img_range_low in range(0,1014544-step,step):
        print(img_range_low)
        img_range_high=img_range_low+step+1
        images = []
        labels = []

        for img_num in range(img_range_low,img_range_high,1):
            img_name=train_data['filename'][img_num]
            img_path = "data/train_data/"+img_name
            if os.path.exists(img_path):
                im = Image.open(img_path).convert('RGB') #转换为灰度图，选项:1,L,P,RGB,RGBA,CMYK,YCbCr,I,F
                imResize = im.resize((299, 299))
                imageData = np.array(imResize)
                images.append(imageData)
                #(filename,extension) = os.path.splitext(img_name)

                #labels.append(train_data['landmark_id'][img_num])
                temp=img_name.split("_")
                temp=temp[3].split(".")
                temp=temp[0].split("[")
                temp=temp[1].split("]")
                temp=temp[0].split(",")

                each_label_list = [0] * 228
                for j in range(0,len(temp)):
                    each_label_list[int(temp[j])-1]=1

                labels.append(each_label_list)

        X = np.array(images)
        y=np.array(labels,dtype=np.float)
        print(X.shape)
        
        if(X.shape[0]>1):
            X = X.reshape(X.shape[0],299,299,3).astype('float32')
            #X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.1,random_state=30)

            #y_train = np_utils.to_categorical(y_train,14951) #矢量编码
            #y_test = np_utils.to_categorical(y_test,14951)

            #model.fit(X,[y,y],epochs=1,batch_size=BATCH_SIZE,verbose=1)
            model.fit(X,y,epochs=1,batch_size=BATCH_SIZE,verbose=1)
            
    ###使用最后的余留的数据
    images = []
    labels = []
    for img_num in range(img_range_high,1014544,1):
        img_name=train_data['filename'][img_num]
        img_path = "data/train_data/"+img_name
        if os.path.exists(img_path):
            im = Image.open(img_path).convert('RGB') #转换为灰度图，选项:1,L,P,RGB,RGBA,CMYK,YCbCr,I,F
            imResize = im.resize((299, 299))
            imageData = np.array(imResize)
            images.append(imageData)
            #(filename,extension) = os.path.splitext(img_name)

            #labels.append(train_data['landmark_id'][img_num])
            temp=img_name.split("_")
            temp=temp[3].split(".")
            temp=temp[0].split("[")
            temp=temp[1].split("]")
            temp=temp[0].split(",")

            each_label_list = [0] * 228
            for j in range(0,len(temp)):
                each_label_list[int(temp[j])-1]=1

            labels.append(each_label_list)

    X = np.array(images)
    y=np.array(labels,dtype=np.float)
    print(X.shape)

    if(X.shape[0]>1):
        X = X.reshape(X.shape[0],299,299,3).astype('float32')

        #model.fit(X,[y,y],epochs=1,batch_size=BATCH_SIZE,verbose=1)
        model.fit(X,y,validation_data=(X_val,y_val),epochs=1,batch_size=BATCH_SIZE,verbose=1)
        
    
        
# images_array = np.array(images)
# labels_array = np.array(labels)
# X = np.array(images)
# y = np.array(labels)

In [6]:
model.save('temp_RIV2_1.h5')